In [26]:
class Transition:
    """The Basic Transition implementation of a state is defined"""

    def __init__(self, InState, InString, OutState, OutString = None):
        """ Defines the basic transition properties and intialized with Argument
            Instate    -   Current state ID
            Instring   -   Transition string input
            OutString  -   Transition output sting
            OutState   -   Next state's ID of the transition
        """

        self.InState = InState
        self.InString = InString
        self.OutString = OutString
        self.OutState = OutState

    def Output(self):
        return [self.OutString, self.OutState]

In [27]:
class State:
    """ This is a class to define the properties of an individual State in a automata"""

    def __init__(self, name, FinalState):
        """ Initailize the State with Arguement:
            ID           -  Unique Identifier for the State (String or Integer)
            StateType    -  Defines the if the State is Terminal State or vice versa (Boolean)
            Transition   -  COntains all transitions from the State (Dictionary)
            """

        self.ID = name
        self.StateType = FinalState
        self.Transition = dict()

    def addTransition(self, InString, OutString, OutState):
        """ Appends a Transition into the class variable 'Transition'
            Arguement of the method:
            Instring    -  Transistion string input 
            Outstring   -  Output string of the transition
            Outstate    -  Next state of the transition
            
            Return:
            None
            """

        if InString in self.Transition.keys():
            #print("Transition with same Instring available")
            self.Transition[InString].append(Transition(self, InString,OutString= OutString, OutState = OutState))
        else:
            #print("New input string Transition")
            self.Transition[InString] = []
            self.Transition[InString].append(Transition(self, InString, OutString = OutString, OutState = OutState))

    def CheckTransition(self, InputAlphabet):

        if InputAlphabet not in self.Transition.keys():
            #print("No Transition as such")
            return [-1]
        elif InputAlphabet in self.Transition.keys():

            Transition_Output_List = []
            for Transit in self.Transition[InputAlphabet]:
                Transition_Output_List.append(Transit.Output())
            
            if '' in self.Transition.keys() and '' != InputAlphabet:
                for epilsonTransit in self.Transition['']:
                    Transition_Output_List.append(epilsonTransit.Output())

            return Transition_Output_List


In [28]:
class Automata:
    """ Basic class to define the automata's structure and its transitions. Further this automata model and be deployed in acceptor or automaton for applications."""

    def __init__(self, StateCount, Alphabet, OutAlphabets, InitialState, FinalStaeCount):
        
        self.StatesCount = StateCount
        self.Alphabet = Alphabet
        self.OutAlphabets = OutAlphabets
        self.InitialState = InitialState
        self.FinalStaeCount = FinalStaeCount

        self.States = dict()
    
    # Methods for building the automata structure

    def AddState(self, ID, Type):

        if self.StatesCount >= len(self.States):
            self.States[ID] = State(ID, Type)
        else:
            print("Exceeding the number of states!!!")

    def AddStateTransition(self, InState, InString, OutString, OutState):

        if InState in self.States.keys() and OutState in self.States.keys():
            In = self.States[InState]
            In.addTransition(InString, OutString, OutState)
            return
        else:
            return -1

    # Create a new Transition object for a state

    def AddsetTransition(self, InState, set ,OutState):
        
        for a in set:
            PCode = self.AddStateTransition(InState, a, a, OutState)
            if PCode == -1:
                print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
                break
        
    def AddsetEpilsonTransition(self, InState, set ,OutState):

        for a in set:
            PCode = self.AddStateTransition(InState, a, '', OutState)
            if PCode == -1:
                print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
                break


    def AddEpilsonTransition(self, InState, OutState):

        PCode = self.AddStateTransition(InState, '', '', OutState)
        if PCode == -1:
            print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')

    def AddEpilsonStringTransition(self, InState, OutString, OutState):

        PCode = self.AddStateTransition(InState, '', OutString, OutState)
        if PCode == -1:
            print(f'No state : {InState} present for the Transition |OR| No state : {OutState} present for the Transition')
        


In [29]:
class Machine:

    def __init__(self, Automata, InputString):
        self.Automata = Automata
        self.IntialState = Automata.InitialState
        self.InputString = InputString

    def Run(self,CurrentState, I = None, String = None):
        #print("Current State: ",CurrentState, " Current Input: ", I)

        if I is None:
            #print("checking for terminal case --->")
            Terminal_Transition_outputs = self.Automata.States[CurrentState].CheckTransition(InputAlphabet = '')
            if len(Terminal_Transition_outputs) > 0 and Terminal_Transition_outputs[0] != -1:
                Terminal_Transition_outputs = self.Automata.States[CurrentState].CheckTransition(InputAlphabet = '')

                Output = []
                for Transit in Terminal_Transition_outputs:
                    if Transit == -1:
                        continue
                    Sub_Output = self.Run(Transit[1])
                    #print(Sub_Output)
                    for S in Sub_Output:

                        S.insert(0,Transit[0])
                        Output.append(S)
                return Output

            
            elif self.Automata.States[CurrentState].StateType == True:
                #print([[True]])
                return [[True]]
            else:
                #print([[False]])
                return [[False]]
            
        else:
            Output = []
            Transition_Outputs = self.Automata.States[CurrentState].CheckTransition(InputAlphabet = I)

            if Transition_Outputs[0] == -1:
                return [[False]]

            else:

                for Transit_Output in Transition_Outputs:
                    
                    if Transit_Output == -1:
                        continue

                    elif len(String) == 0:
                        Sub_Transition_Output = self.Run(Transit_Output[1], I = None, String= None)
                    else:
                        Sub_Transition_Output = self.Run(Transit_Output[1], I = String[0], String= String[1:])
                    
                    for S in Sub_Transition_Output:
                        S.insert(0,Transit_Output[0])
                        Output.append(S) 
                return Output
    
    def Machine_Automata_Output(self):

        print("Initiating machine")
        OutputList = self.Run(self.IntialState, I= self.InputString[0], String=self.InputString[1:])
        print("Completing the Generation...")
        for Output in OutputList:
            if True in Output:
                Output.remove(True)
                Generated_String = "".join(Output)
                print("Input String: " + self.InputString)
                print("Generated Output String -->" + Generated_String)
                break



### **NFA EXAMPLE**

In [30]:
AutoEx_1 = Automata(StateCount=3, Alphabet=set('01'), OutAlphabets= None, InitialState='a', FinalStaeCount=1)

# Adding States
AutoEx_1.AddState(ID='a', Type= False)
AutoEx_1.AddState(ID='b', Type= True)
AutoEx_1.AddState(ID='c', Type= True)

# Adding Transitions
AutoEx_1.AddStateTransition(InState='a', InString = '1', OutString = '0', OutState= 'b')
AutoEx_1.AddStateTransition(InState='a', InString = '1', OutString = '1', OutState= 'c')
AutoEx_1.AddStateTransition(InState='a', InString = '0', OutString = '1', OutState= 'c')
AutoEx_1.AddEpilsonTransition(InState='a', OutState= 'c')
AutoEx_1.AddEpilsonStringTransition(InState='a', OutString='1',OutState= 'c')

### **FST AUTOMATA**

In [31]:
A = Automata(StateCount=12, Alphabet=set('abcdefghijklmnopqrstuvwxyz'), OutAlphabets= None, InitialState='q0', FinalStaeCount=1)

#Adding States
A.AddState(ID='q0',Type=False)
A.AddState(ID='q1',Type=False)
A.AddState(ID='q_ing',Type=False)
A.AddState(ID='q_drop_E',Type=False)
A.AddState(ID='q_EE',Type=False)
A.AddState(ID='q_IE',Type=False)
A.AddState(ID='q_VOWS',Type=False)
A.AddState(ID='q_VOWS2',Type=False)
A.AddState(ID='q_CONS',Type=False)
A.AddState(ID='q_CONS+E',Type=False)
A.AddState(ID='q_CONS_VOWS-E',Type=False)
A.AddState(ID='q_EOW',Type=True)

A2Z=set('abcdefghijklmnopqrstuvwxyz')
VOWS=set('aieou')
CONS=A2Z-VOWS
U=set('u')
E=set('e')
I=set('i')
PT=set('pt')
NR=set('nr')

In [32]:
#Adding Transitions
#AddsetTransition(self, InState, set ,OutState)
A.AddsetTransition('q0', VOWS , 'q_VOWS')
A.AddsetTransition('q_VOWS', VOWS, 'q_VOWS2')
A.AddsetTransition('q_VOWS2',CONS,'q_ing')
#AddEpilsonStringTransition(self, InState, OutString, OutState)
A.AddEpilsonStringTransition('q_ing','ing','q_EOW')

In [33]:
##check
#AddsetTransition(self, InState, set ,OutState)
A.AddsetTransition('q0', A2Z ,'q1')
A.AddsetTransition('q1', A2Z ,'q1')
A.AddsetTransition('q1', CONS.union(U) ,'q_drop_E')
#AddsetEpilsonTransition(self, InState, set ,OutState)
A.AddsetEpilsonTransition('q_drop_E',E,'q_ing')
A.AddsetTransition('q1', E ,'q_EE')
A.AddsetTransition('q_EE', E ,'q_ing') ##########################
A.AddsetEpilsonTransition('q1',I,'q_IE')
#AddStateTransition(self, InState, InString, OutString, OutState)
A.AddStateTransition('q_IE', 'e','y','q_ing')
A.AddsetTransition('q1',CONS,'q_CONS')
A.AddsetTransition('q1',VOWS,'q_VOWS')

In [34]:
#AddsetTransition(self, InState, set ,OutState)
A.AddsetTransition('q0', CONS ,'q_CONS')
A.AddsetTransition('q_CONS', E ,'q_CONS+E')
A.AddsetTransition('q_CONS+E',A2Z-PT-VOWS,'q_ing')
#AddStateTransition(self, InState, InString, OutString, OutState
A.AddStateTransition('q_cons+E','p','pp','q_ing')
A.AddStateTransition('q_cons+E','t','tt','q_ing')
A.AddsetTransition('q_CONS', VOWS-E ,'q_CONS_VOWS-E')
A.AddsetTransition('q_CONS_VOWS-E',A2Z-NR-PT-VOWS,'q_ing')
A.AddStateTransition('q_CONS_VOWS-E','p','pp','q_ing')
A.AddStateTransition('q_CONS_VOWS-E','n','nn','q_ing')
A.AddStateTransition('q_CONS_VOWS-E','r','rr','q_ing')
A.AddStateTransition('q_CONS_VOWS-E','t','tt','q_ing')

### **TESTING WITH INPUTS**

In [35]:
O = Machine(A,"bite")
A.InitialState
O.Run(A.InitialState, I=O.InputString[0],String=O.InputString[1:])

[['b', 'i', 't', 'e', False],
 ['b', 'i', 't', 'e', False],
 ['b', 'i', 't', 'e', False],
 ['b', 'i', 't', '', 'ing', True],
 ['b', 'i', 't', 'e', False],
 ['b', '', False],
 ['b', 'i', False],
 ['b', 'i', 'tt', False]]

In [36]:
O.Machine_Automata_Output()

Initiating machine
Completing the Generation...
Input String: bite
Generated Output String -->biting
